In [1]:
from os import listdir

files = sorted(listdir('data'))
if '.DS_Store' in files:
    files.remove('.DS_Store') # annoying Mac feature

tickers = [file[:-4] for file in files] # remove '.csv' extension

In [3]:
from pathlib import Path
import csv

paths = [Path('data') / file for file in files] # using class 'Path' ensures proper processing for both Unix and Windows

data = [] # will become list of lists of OHLC tuples

for path in paths:
    
    with open(path, newline='') as file:
        
        reader = csv.reader(file)
        line = next(reader)
        while line[0] != '2019-09-12': # 77th trading day before 2020/1/2 (indicator requires 77 days of historical data)
            line = next(reader)
        
        list = []
        while line[0] != '2024-01-02':
            o = float(line[1])
            h = float(line[2])
            l = float(line[3])
            c = float(line[5]) # (adjusted close after dividends)
            list.append((round(o,2), round(h,2), round(l,2), round(c,2)))
            line = next(reader)

        data.append(list)

In [5]:
def strategy(list):

    highs = [ohlc[1] for ohlc in list]
    lows = [ohlc[2] for ohlc in list]

    base = [] # "kijun-sen"
    for i in range(77, len(list)):
        high = max(highs[i-25:i+1])
        low = min(lows[i-25:i+1])
        base.append(round((high+low)/2, 2))

    conv = [] # "tenkan-sen"
    for i in range(77, len(list)):
        high = max(highs[i-8:i+1])
        low = min(lows[i-8:i+1])
        conv.append(round((high+low)/2, 2))

    lead = [] # "senkou span B" (lower edge of a bullish cloud, upper of bearish)
    for i in range(77, len(list)):
        high = max(highs[i-77:i-25])
        low = min(lows[i-77:i-25])
        lead.append(round((high+low)/2, 2))

    closes = [ohlc[3] for ohlc in list]
    closes = closes[77:] # now contains data from 2020/1/2 to 2023/12/29, like the indicator

    ##################################
    
    cl_abv_base = closes[0] > base[0]

    in_trade = False
    periods = 0
    entry = 0

    results = [] # percent profit/loss
    lengths = [] # length (trading, not calendar days)
        
    for i in range(1, len(closes)): # simulate trading
            
        if (not cl_abv_base) and (closes[i] > base[i]) and (closes[i] > conv[i]) and (closes[i] > lead[i]) and (not in_trade): # entry

            cl_abv_base = True
            in_trade = True
            entry = closes[i]

        if closes[i] < base[i]:

            cl_abv_base = False

        if in_trade:

            if closes[i] < conv[i]: # exit

                in_trade = False
                percent = round((closes[i]-entry)*100/entry, 2)
                results.append(percent)
                lengths.append(periods)
                periods = 0
                entry = 0

            else:

                periods = periods + 1

    if in_trade and periods > 0: # on 2023/12/29
        
        percent = round((closes[len(closes)-1]-entry)*100/entry, 2)
        results.append(percent)
        lengths.append(periods)

    return results, lengths

In [7]:
combined_results = []
combined_lengths = []

for i in range(len(tickers)):
        
    results, lengths = strategy(data[i])

    for j in range(len(results)):

        combined_results.append(results[j])
        combined_lengths.append(lengths[j])

    if len(results) != 0:
        
        avg_result = round(sum(results)/len(results), 2)
        avg_length = round(sum(lengths)/len(lengths))
        
        print(tickers[i] + ': Avg. Result = ' + str(avg_result) + '%; Avg. Length = ' +
              str(avg_length) + ' Days; # Trades = ' + str(len(results)))

    else:

        print(tickers[i] + ': * NO TRADES MADE *')

AAPL: Avg. Result = 0.06%; Avg. Length = 5 Days; # Trades = 25
AMGN: Avg. Result = 0.74%; Avg. Length = 4 Days; # Trades = 4
AMZN: Avg. Result = 0.25%; Avg. Length = 8 Days; # Trades = 28
AXP: Avg. Result = 0.24%; Avg. Length = 4 Days; # Trades = 16
BA: Avg. Result = -0.36%; Avg. Length = 8 Days; # Trades = 26
CAT: Avg. Result = 5.5%; Avg. Length = 6 Days; # Trades = 7
CRM: Avg. Result = 1.86%; Avg. Length = 8 Days; # Trades = 26
CSCO: Avg. Result = -0.71%; Avg. Length = 1 Days; # Trades = 1
CVX: Avg. Result = 2.61%; Avg. Length = 2 Days; # Trades = 1
DIS: Avg. Result = 2.42%; Avg. Length = 7 Days; # Trades = 14
DOW: Avg. Result = 1.85%; Avg. Length = 9 Days; # Trades = 1
GS: Avg. Result = -1.29%; Avg. Length = 3 Days; # Trades = 5
HD: Avg. Result = 0.33%; Avg. Length = 4 Days; # Trades = 5
HON: Avg. Result = 1.4%; Avg. Length = 4 Days; # Trades = 4
IBM: Avg. Result = -0.29%; Avg. Length = 1 Days; # Trades = 1
INTC: Avg. Result = 1.11%; Avg. Length = 6 Days; # Trades = 10
JNJ: Avg. Res

In [9]:
avg_result = round(sum(combined_results)/len(combined_results), 2)
avg_length = round(sum(combined_lengths)/len(combined_lengths))

print('COMBINED: Avg. Result = ' + str(avg_result) + '%; Avg. Length = ' +
      str(avg_length) + ' Days; # Trades = ' + str(len(combined_results)))

COMBINED: Avg. Result = 0.36%; Avg. Length = 5 Days; # Trades = 301
